In [1]:
import pandas as pd
df = pd.read_csv(r'Data Preprocessing\bank_combined.csv')

In [2]:
df['term deposits'] = df['y']
df = df.drop(columns=['y'])

In [3]:
from llama_index.core import Document
documents = []

for _, row in df.iterrows():
    # Combine all relevant columns into a single string
    text = f"""
    Age: {row['age']}, 
    Job: {row['job']}, 
    Marital: {row['marital']}, 
    Education: {row['education']}, 
    Default: {row['default']}, 
    Balance: {row['balance']}, 
    Housing: {row['housing']}, 
    Loan: {row['loan']}, 
    Contact: {row['contact']}, 
    Day: {row['day']}, 
    Month: {row['month']}, 
    Duration: {row['duration']}, 
    Campaign: {row['campaign']}, 
    Pdays: {row['pdays']}, 
    Previous: {row['previous']}, 
    Poutcome: {row['poutcome']}, 
    term deposits: {row['term deposits']}
    """
    
    # Create Document
    documents.append(Document(text=text))

In [4]:
from llama_index.llms.groq import Groq
import os
from dotenv import load_dotenv
from llama_index.core import Settings

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise EnvironmentError("GROQ_API_KEY is missing. Please set it in your .env file.")

llm = Groq(model="allam-2-7b", api_key=GROQ_API_KEY)
Settings.llm = llm

d:\University\knowledge representation\CRM GRAPHRAG KRR\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Use a local (non-OpenAI) embedding model so no OPENAI_API_KEY is required
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [6]:
llm.complete("Hello, world!").text

"Hello there! As an AI assistant, I am always ready to chat, answer your questions, or assist you in any way I can. How may I help you today? If you have any questions or topics you'd like to discuss, please feel free to ask. ]{: .notice} I'm here to provide information, assist with tasks, or simply engage in conversation. Don't hesitate to ask me anything, and I'll do my best to respond appropriately. "

In [7]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from typing import Any, List, Callable, Optional, Union, Dict
from IPython.display import Markdown, display

from llama_index.core.async_utils import run_jobs
from llama_index.core.indices.property_graph.utils import (
    default_parse_triplets_fn,
)
from llama_index.core.graph_stores.types import (
    EntityNode,
    KG_NODES_KEY,
    KG_RELATIONS_KEY,
    Relation,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.default_prompts import (
    DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
)
from llama_index.core.schema import TransformComponent, BaseNode
from llama_index.core.bridge.pydantic import BaseModel, Field
class GraphRAGExtractor(TransformComponent):
    """Extract triples from a graph using Groq."""

    llm: LLM
    extract_prompt: PromptTemplate
    parse_fn: Callable
    num_workers: int
    max_paths_per_chunk: int

    def __init__(
        self,
        llm: Optional[LLM] = None,
        extract_prompt: Optional[Union[str, PromptTemplate]] = None,
        parse_fn: Callable = default_parse_triplets_fn,
        max_paths_per_chunk: int = 10,
        num_workers: int = 1,
    ) -> None:
        from llama_index.core import Settings

        if isinstance(extract_prompt, str):
            extract_prompt = PromptTemplate(extract_prompt)

        super().__init__(
            llm=llm or Settings.llm,
            extract_prompt=extract_prompt or DEFAULT_KG_TRIPLET_EXTRACT_PROMPT,
            parse_fn=parse_fn,
            num_workers=num_workers,
            max_paths_per_chunk=max_paths_per_chunk,
        )

    @classmethod
    def class_name(cls) -> str:
        return "GraphExtractor"

    def __call__(self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any) -> List[BaseNode]:
        return asyncio.run(self.acall(nodes, show_progress=show_progress, **kwargs))

    async def _aextract(self, node: BaseNode) -> BaseNode:
        """Extract triples using Groq LLM."""

        assert hasattr(node, "text")

        # Prepare text for model
        text = node.get_content(metadata_mode="llm")

        # Render prompt (Groq requires full prompt as a single string)
        prompt_str = self.extract_prompt.format(
            text=text,
            max_knowledge_triplets=self.max_paths_per_chunk
        )

        try:
            # ----------- 👇 UPDATED FOR GROQ -----------
            llm_response = await self.llm.acomplete(prompt_str)
            output_text = llm_response.text
            # -------------------------------------------

            entities, entities_relationship = self.parse_fn(output_text)

        except Exception as e:
            print("Error in Groq call:", e)
            entities = []
            entities_relationship = []

        # ======================
        # BUILD ENTITY NODES
        # ======================
        existing_nodes = node.metadata.pop(KG_NODES_KEY, [])
        existing_relations = node.metadata.pop(KG_RELATIONS_KEY, [])
        metadata = node.metadata.copy()

        for entity, entity_type, description in entities:
            metadata["entity_description"] = description
            entity_node = EntityNode(
                name=entity,
                label=entity_type,
                properties=metadata
            )
            existing_nodes.append(entity_node)

        # ======================
        # BUILD RELATION TRIPLES
        # ======================
        metadata = node.metadata.copy()

        for subj, obj, rel, description in entities_relationship:
            subj_node = EntityNode(name=subj, properties=metadata)
            obj_node = EntityNode(name=obj, properties=metadata)
            metadata["relationship_description"] = description

            rel_node = Relation(
                label=rel,
                source_id=subj_node.id,
                target_id=obj_node.id,
                properties=metadata
            )

            existing_nodes.extend([subj_node, obj_node])
            existing_relations.append(rel_node)

        node.metadata[KG_NODES_KEY] = existing_nodes
        node.metadata[KG_RELATIONS_KEY] = existing_relations

        return node

    async def acall(self, nodes: List[BaseNode], show_progress: bool = False, **kwargs: Any) -> List[BaseNode]:
        jobs = [self._aextract(node) for node in nodes]

        return await run_jobs(
            jobs,
            workers=self.num_workers,
            show_progress=show_progress,
            desc="Extracting paths from text"
        )


In [8]:
import re
from llama_index.core.graph_stores import SimplePropertyGraphStore
import networkx as nx
from graspologic.partition import hierarchical_leiden

from llama_index.core.llms import ChatMessage


class GraphRAGStore(SimplePropertyGraphStore):
    community_summary = {}
    max_cluster_size = 5

    def __init__(self, llm: LLM = None, **kwargs):
        super().__init__(**kwargs)
        # Use provided LLM or fallback to global Settings
        self.llm = llm or Settings.llm

    def generate_community_summary(self, text):
        """Generate summary for a given text using an LLM."""
        prompt = (
            "You are provided with a set of customer relationships from a bank marketing dataset. "
            "Each relationship is represented as 'Customer1 -> Customer2 -> relationship_description'. "
            "Create a coherent summary that highlights patterns and clusters among customers, focusing on "
            "age, job, marital status, education, balance, housing, loans, and term deposits. "
            "Emphasize which groups might be more likely to subscribe to term deposits or show similar financial behavior.\n\n"
            f"{text}"
        )
        response = self.llm(prompt)
        # LlamaIndex LLM returns a string directly
        clean_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return clean_response

    def build_communities(self):
        """Builds communities from the graph and summarizes them."""
        nx_graph = self._create_nx_graph()
        community_hierarchical_clusters = hierarchical_leiden(
            nx_graph, max_cluster_size=self.max_cluster_size
        )
        community_info = self._collect_community_info(
            nx_graph, community_hierarchical_clusters
        )
        self._summarize_communities(community_info)

    def _create_nx_graph(self):
        """Converts internal graph representation to NetworkX graph using bank dataset columns."""
        nx_graph = nx.Graph()
        # Add nodes with meaningful labels
        for node in self.graph.nodes.values():
            node_label = (
                f"Age:{node.properties.get('age','')}, "
                f"Job:{node.properties.get('job','')}, "
                f"Marital:{node.properties.get('marital','')}, "
                f"Education:{node.properties.get('education','')}"
            )
            nx_graph.add_node(str(node_label))
        # Add edges with relationship info
        for relation in self.graph.relations.values():
            source = relation.source_id
            target = relation.target_id
            edge_desc = (
                f"Balance:{relation.properties.get('balance','')}, "
                f"Housing:{relation.properties.get('housing','')}, "
                f"Loan:{relation.properties.get('loan','')}, "
                f"Term Deposits:{relation.properties.get('term deposits','')}"
            )
            nx_graph.add_edge(source, target, relationship=relation.label, description=edge_desc)
        return nx_graph

    def _collect_community_info(self, nx_graph, clusters):
        """Collect detailed information for each node based on their community."""
        community_mapping = {item.node: item.cluster for item in clusters}
        community_info = {}
        for item in clusters:
            cluster_id = item.cluster
            node = item.node
            if cluster_id not in community_info:
                community_info[cluster_id] = []

            for neighbor in nx_graph.neighbors(node):
                if community_mapping[neighbor] == cluster_id:
                    edge_data = nx_graph.get_edge_data(node, neighbor)
                    if edge_data:
                        detail = f"{node} -> {neighbor} -> {edge_data['relationship']} -> {edge_data['description']}"
                        community_info[cluster_id].append(detail)
        return community_info

    def _summarize_communities(self, community_info):
        """Generate and store summaries for each community."""
        for community_id, details in community_info.items():
            details_text = "\n".join(details) + "."  # Ensure it ends with a period
            self.community_summary[community_id] = self.generate_community_summary(details_text)

    def get_community_summaries(self):
        """Returns the community summaries, building them if not already done."""
        if not self.community_summary:
            self.build_communities()
        return self.community_summary

In [9]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.llms import LLM


class GraphRAGQueryEngine(CustomQueryEngine):
    graph_store: GraphRAGStore
    llm: LLM

    def custom_query(self, query_str: str) -> str:
        """Process all community summaries to generate answers to a specific query."""
        community_summaries = self.graph_store.get_community_summaries()
        community_answers = [
            self.generate_answer_from_summary(community_summary, query_str)
            for _, community_summary in community_summaries.items()
        ]

        final_answer = self.aggregate_answers(community_answers)
        return final_answer

    def generate_answer_from_summary(self, community_summary, query):
        """Generate an answer from a community summary based on a given query using LLM."""
        prompt = (
            f"Given the community summary: {community_summary}, "
            f"how would you answer the following query? Query: {query}"
        )
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content="I need an answer based on the above information.",
            ),
        ]
        
        response = self.llm.complete(messages)
        cleaned_response = re.sub(r"^assistant:\s*", "", str(response)).strip()
        return cleaned_response

    def aggregate_answers(self, community_answers):
        """Aggregate individual community answers into a final, coherent response."""
        # intermediate_text = " ".join(community_answers)
        prompt = "Combine the following intermediate answers into a final, concise response."
        messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(
                role="user",
                content=f"Intermediate answers: {community_answers}",
            ),
        ]
        final_response = self.llm.chat(messages)
        cleaned_final_response = re.sub(
            r"^assistant:\s*", "", str(final_response)
        ).strip()
        return cleaned_final_response

In [10]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

In [11]:
KG_TRIPLET_EXTRACT_TMPL_BANK = """
-Goal-
Given a customer record from a bank marketing dataset, identify all entities and their entity types from the data and all relationships among the identified entities.
Extract up to {max_knowledge_triplets} entity-relation triplets.

-Steps-
1. Identify all entities. For each entity, extract the following information:
- entity_name: Name of the entity (e.g., a customer ID, or attribute name like Job, Marital Status)
- entity_type: Type of the entity (e.g., Customer, Attribute)
- entity_description: Comprehensive description of the entity's attribute value or customer characteristics

2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are clearly related to each other.
For each pair of related entities, extract the following information:
- source_entity: name of the source entity
- target_entity: name of the target entity
- relation: relationship between source_entity and target_entity
- relationship_description: explanation as to why the source entity and the target entity are related (e.g., correlation, dependency, influence on term deposit subscription)

3. Output Formatting:
- Return the result in valid JSON format with two keys: 'entities' (list of entity objects) and 'relationships' (list of relationship objects).
- Exclude any text outside the JSON structure.
- If no entities or relationships are identified, return empty lists: { "entities": [], "relationships": [] }.

-An Output Example-
{
  "entities": [
    {
      "entity_name": "Customer 101",
      "entity_type": "Customer",
      "entity_description": "A 35-year-old married customer with a tertiary education and a balance of 5000, has a housing loan but no personal loan."
    },
    {
      "entity_name": "Job",
      "entity_type": "Attribute",
      "entity_description": "The customer works as a technician."
    },
    {
      "entity_name": "Housing Loan",
      "entity_type": "Attribute",
      "entity_description": "The customer has an active housing loan."
    }
  ],
  "relationships": [
    {
      "source_entity": "Customer 101",
      "target_entity": "Housing Loan",
      "relation": "has",
      "relationship_description": "Customer 101 has a housing loan, which may influence their likelihood to subscribe to a term deposit."
    },
    {
      "source_entity": "Customer 101",
      "target_entity": "Job",
      "relation": "works_as",
      "relationship_description": "Customer 101 works as a technician, which may influence their income and balance."
    }
  ]
}

-Real Data-
######################
text: {text}
######################
output:
"""


In [ ]:
import json
import ast
from json import JSONDecodeError

try:
    import json5  # lenient JSON parser
except ImportError:  # pragma: no cover
    json5 = None

def _extract_json_block(text: str) -> str | None:
    start = text.find("{")
    if start == -1:
        return None

    stack = 0
    in_string = False
    escape = False

    for idx in range(start, len(text)):
        ch = text[idx]
        if ch == "\\" and not escape:
            escape = True
            continue
        if ch == '"' and not escape:
            in_string = not in_string
        escape = False

        if in_string:
            continue

        if ch == "{":
            stack += 1
        elif ch == "}":
            stack -= 1
            if stack == 0:
                return text[start : idx + 1]
    return None


def _load_lenient_json(json_str: str):
    try:
        return json.loads(json_str)
    except JSONDecodeError:
        pass

    try:
        decoder = json.JSONDecoder(strict=False)
        data, _ = decoder.raw_decode(json_str)
        return data
    except JSONDecodeError:
        pass

    if json5 is not None:
        try:
            return json5.loads(json_str)
        except Exception:
            pass

    try:
        return ast.literal_eval(json_str)
    except (SyntaxError, ValueError):
        return None


def parse_fn(response_str: str):
    json_str = _extract_json_block(response_str)
    if not json_str:
        return [], []

    json_str = json_str.replace("\r", " ").replace("\t", " ")
    data = _load_lenient_json(json_str)
    if data is None:
        print("Error parsing JSON: could not repair output")
        return [], []

    entities = [
        (
            entity.get("entity_name", "Unknown Entity"),
            entity.get("entity_type", "Attribute"),
            entity.get("entity_description", ""),
        )
        for entity in data.get("entities", [])
    ]

    relationships = [
        (
            relation.get("source_entity", ""),
            relation.get("target_entity", ""),
            relation.get("relation", "related_to"),
            relation.get("relationship_description", ""),
        )
        for relation in data.get("relationships", [])
    ]

    return entities, relationships


kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL_BANK,
    max_paths_per_chunk=1,
    parse_fn=parse_fn,
    num_workers=1,
)

In [13]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex(
    nodes=nodes,
    property_graph_store=GraphRAGStore(llm=llm),
    kg_extractors=[kg_extractor],
    show_progress=True,
)

Extracting paths from text:   0%|          | 9/49732 [01:16<147:54:28, 10.71s/it]

Error parsing JSON: Expecting ',' delimiter: line 184 column 1 (char 5524)


Extracting paths from text:   0%|          | 10/49732 [01:29<155:30:25, 11.26s/it]

Error parsing JSON: Expecting ',' delimiter: line 178 column 1 (char 5093)


Extracting paths from text:   0%|          | 12/49732 [01:52<160:20:01, 11.61s/it]

...

Error in Groq call: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499429, Requested 965. Please try again in 1m8.083199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Retrying llama_index.llms.openai.base.OpenAI._achat in 1.5785880291721845 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499422, Requested 963. Please try again in 1m6.527999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499412, Requested 961. Please try again in 1m4.4544s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceede

Error in Groq call: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499412, Requested 963. Please try again in 1m4.8s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Retrying llama_index.llms.openai.base.OpenAI._achat in 1.2217425729754268 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499405, Requested 961. Please try again in 1m3.2448s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499397, Requested 962. Please try again in 1m2.0352s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}.

Error in Groq call: Error code: 429 - {'error': {'message': 'Rate limit reached for model `allam-2-7b` in organization `org_01jvqbbz09fatbtkjc8p1g3zjy` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499398, Requested 961. Please try again in 1m2.0352s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
